In [31]:
#MeteCritic Pagescraper
import requests
import pandas as pd
from bs4 import BeautifulSoup



In [ ]:
#MeteCritic URL Retriever

url_dict = {'url':[]} # Data Structure

def webpage(pageNum, year): #function that navigates the Metacritic SRP(Search Results Pages) based on the page number and the year
          
    url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected='+ str(year) +'&distribution=&sort=desc&view=condensed&page='+ str(pageNum)
    userAgent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=userAgent)
    return response

def numberPages(response): # Helper Function that determines how many pages are in a SRP to know how many times to run scrapper function
    soup = BeautifulSoup(response.text, 'html.parser')
    pages = soup.find_all('li', {"class":"page last_page"})
    pagesCleaned = pages[0].find('a', {"class":"page_num"})
    return (pagesCleaned.text)

def scrapper(num_loops, content):
    tblnum = 0
    while tblnum < num_loops:
        #getting game url
        table_rows = content[tblnum].find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            for a in td[1].find_all('a', {"class":"title"} ,href=True):
                url_dict['url'].append(a['href'])
        tblnum += 1

def pages(lastPageNum, year): #Function that returns the html(code) and initiates the URL Retriever
    currentPage = 0
    while currentPage < int(lastPageNum):
        url = url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected='+ str(year) +'&distribution=&sort=desc&view=condensed&page=' + str(currentPage)
        userAgent = {'User-agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=userAgent)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find_all('table')
        num_loops = len(content)
        scrapper(num_loops, content)
        currentPage += 1
        print(currentPage)
        time.sleep(6)

#Main code
years = list(range(2010, 2020))
for year in years:
    if(year < 2000):
        pages(1, year)
        time.sleep(5)
        print(year)
    else:
        numPage = (numberPages(webpage(0, year)))
        pages(int(numPage), year)
        time.sleep(5)
        print(year)
url_data = (pd.DataFrame.from_dict(url_dict))

In [32]:
final_list = []
def page_scraper(movie_url):
    #Getting the URL and setting up the soup page to be scraped
    url = "https://www.metacritic.com" + movie_url
    user_agent = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0'}
    response  = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Getting the Title of the movie
    product_title = soup.find("div", class_ = "product_page_title oswald")
    if(product_title is not None):
        movie_title = product_title.find("h1")
        movie_title = movie_title.text.strip()




            #Getting the Release Date of the film

    release_date = soup.find("span", class_ = "release_year lighter")
    if(release_date is not None):
        release_date = str(release_date.text.strip())
    else:
        release_date = "N/A"

                #Getting the Metascore
    meta_score = soup.find("span", class_="metascore_w header_size movie positive")
    if (meta_score is not None):
        meta_score = meta_score.text.strip()
        meta_score = int(meta_score)
    else:
        meta_score = soup.find("span",class_="metascore_w header_size movie negative")
        if (meta_score is not None):
            meta_score = meta_score.text.strip()
            meta_score = int(meta_score)
        else:
            meta_score = soup.find("span",class_="metascore_w header_size movie mixed") 
            if(meta_score is not None):
                meta_score = meta_score.text.strip()
                meta_score = int(meta_score)
            else:
                meta_score = soup.find("span",class_="metascore_w header_size movie positive perfect")
                meta_score = meta_score.text.strip()
                meta_score = int(meta_score)
            
            #Getting Number of metascore Vote    
    meta_number = soup.find("div",class_="ms_wrapper")
    meta_number = meta_number.find("span",class_="based_on")
    if(meta_score is not None):
        meta_number = meta_number.text.strip()
        meta_number = [int(s) for s in meta_number.split() if s.isdigit()]
        meta_number = int(meta_number[0])
    else:
        meta_number = 0

                #Gtting the Audience Metascore
    meta_audience = soup.find("span",class_ = "metascore_w user larger movie positive")
    if (meta_audience is not None):
        meta_audience = meta_audience.text.strip()
        meta_audience_vote = float(meta_audience)*10
    else:
        meta_audience = soup.find("span",class_="metascore_w user larger movie negative")
        if (meta_audience is not None):
            meta_audience = meta_audience.text.strip()
            meta_audience_vote = float(meta_audience)*10
        else:
            meta_audience = soup.find("span",class_="metascore_w user larger movie mixed")
            if (meta_audience is not None):
                meta_audience = meta_audience.text.strip()
                meta_audience_vote = float(meta_audience)*10
            else:
                meta_audience_vote = meta_score


                #Getting the Audience Metascore Number:
    meta_audience_number = soup.find("div",class_ = "user_score_summary")
    meta_audience_number = meta_audience_number.find("span",class_ = "based_on")
    if (meta_audience_number is not None):
        meta_audience_number = meta_audience_number.text.strip()
        meta_audience_number = [int(s) for s in meta_audience_number.split() if s.isdigit()]
        meta_audience_number = int(meta_audience_number[0])
    else :
        meta_audience_number = meta_number
        
    list = [movie_title,release_date,meta_score,meta_number,meta_audience_vote,meta_audience_number]    
    
    final_list.append(list)
    
       
        
        
for i in range(2175,5881):
    page_scraper(movie['url'][i])


In [40]:
df2 = pd.DataFrame({
    'Title': [str(x[0]) for x in final_list],
     'Year': [(x[1]) for x in final_list],
     'mc_av':[(x[2]) for x in final_list],
     'mc_no':[(x[3]) for x in final_list],
     'ma_av':[int(x[4]) for x in final_list],
     'ma_no':[(x[5]) for x in final_list]
     
    })

In [20]:
df

,Title,Year,mc_av,mc_no,ma_av,ma_no
0,The Social Network,2010,95,42,82,2171
1,Carlos,2010,94,21,79,78
2,Toy Story 3,2010,92,39,89,1953
3,Winter's Bone,2010,90,38,73,512
4,A Prophet,2010,90,31,85,215
...,...,...,...,...,...,...
495,Buck,2011,76,31,67,21
496,Silent Souls,2011,76,15,83,4
497,Margin Call,2011,76,38,72,168
498,World on a Wire (1973),2011,76,7,75,10


In [38]:

df1

,Title,Year,mc_av,mc_no,ma_av,ma_no
0,The Social Network,2010,95,42,82,2171
1,Carlos,2010,94,21,79,78
2,Toy Story 3,2010,92,39,89,1953
3,Winter's Bone,2010,90,38,73,512
4,A Prophet,2010,90,31,85,215
...,...,...,...,...,...,...
2169,I Will Follow You Into the Dark,2013,17,9,58,11
2170,Girls Against Boys,2013,17,8,37,17
2171,Evidence,2013,14,8,49,13
2172,Nothing Left to Fear,2013,12,7,35,18


In [41]:
df2

,Title,Year,mc_av,mc_no,ma_av,ma_no
0,Mr. Turner,2014,94,44,69,152
1,Leviathan,2014,92,34,73,259
2,Ida,2014,91,35,75,211
3,Big Men,2014,90,11,74,9
4,The Overnighters,2014,89,20,75,14
...,...,...,...,...,...,...
3701,Replicas,2019,19,15,47,86
3702,Polar,2019,19,12,64,226
3703,The Fanatic,2019,18,19,44,29
3704,Unplanned,2019,10,7,94,1989


In [49]:
df_final = pd.concat([df1,df2],axis=0).reset_index()
df_final

,index,Title,Year,mc_av,mc_no,ma_av,ma_no
0,0,The Social Network,2010,95,42,82,2171
1,1,Carlos,2010,94,21,79,78
2,2,Toy Story 3,2010,92,39,89,1953
3,3,Winter's Bone,2010,90,38,73,512
4,4,A Prophet,2010,90,31,85,215
...,...,...,...,...,...,...,...
5875,3701,Replicas,2019,19,15,47,86
5876,3702,Polar,2019,19,12,64,226
5877,3703,The Fanatic,2019,18,19,44,29
5878,3704,Unplanned,2019,10,7,94,1989


In [50]:
df_final.to_csv('/Users/Amin/Desktop/mojo.csv')